# Fourier Tutorial

Fourier analysis tutorial for NeuBeh/PBIO 545, Winter 2003
- Created 1/03 Mike Shadlen
- Revisions:
  - GDF: cleaned up some typos
  - RK: 6/2010
  - MLW: 6/2018: Python translation

The fourier transform is probably the most important transform in applied
math. It takes a function of time or space (or whatever) and expresses it
as a function of frequency. The goal of this tutorial is make you
comfortable with what a fourier transform is and why it is useful. We
will focus on two topics: (1) definition and properties of the fourier
transform, (2) convolution, and (3) the connection between the two. 


In [ ]:
%matplotlib inline
import numpy as np
from numpy import sin, cos, exp, pi
from numpy.fft import fft, ifft, fftshift, ifftshift, fftfreq
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML


## Part I. The discrete fourier transform

A transform takes a value and makes it into a new value. Log is a
familiar example. Or, we might say that it takes a list of values and
makes them into a new list of values.  When we think about the fourier
transform, we consider a list of intensities at points in time (or space)
and transform them to a list of "intensities" at different frequencies.
Thinking about a signal as a collection of time intensities, each
associated with its own time point, is simple enough. Thinking about a
signal as a list of intensities, each associated with its own sinusoidal
frequency, is also convenient. You do it every day when you adjust the
bass and treble on your stereo -- or better yet, if you adjust knobs on
an equalizer. You can imagine that describing a signal in terms of its
frequencies can be convenient in certain settings. That's what the
fourier transform is for. 

I want to emphasize from the start that the fourier transform (FT, for
short) does not change the signal; it just represents it differently. To
be concrete, suppose there is some electrical signal that we want to
describe over the phone to a friend. We could list the voltages at each
time or we could list the amplitudes and phases of each of the
frequencies that comprise the signal. It's the same signal, just a
different list of numbers. 

Often, when we describe signals, we don't think of them as lists of
numbers but rather as functions of time. Of course, the function can be
viewed as a way to generate a list of values at each time point. We
can also use a function to describe the list of "intensities" at each
frequency in a fourier transform. The two functions are called fourier
transform pairs. For example, suppose $s(t)$ is some function of time (it
returns a value at each time point). And suppose that you could describe
the same signal by "intensities" of frequency ($\omega$) using a function, $S(\omega)$.
Then, we say that $s(t)$ and $S(\omega)$ are a Fourier transform pair. Or we say
that $S(w)$ is the Fourier transform of $s(t)$, or that $s(t)$ is the inverse
Fourier transform of $S(\omega)$. 

Before we look at FTs, let's get comfortable with a few illustrative
functions. We will focus on discrete functions. What is meant by this is
that the argument of the function, say time, has discrete values. We're
used to thinking of t occupying some continuous interval on a the real
number line, but I'm asking you to think of time as integers, or really
integers multiplied by some constant or scaling factor.



In [ ]:
# Define a discrete time axis. The spacing is dt
dt = .01  # our units are seconds. So each integer time point is scaled to dt sec
tmax = 1
t = np.arange(0, tmax, dt)

#Notice that we are sampling time at a rate of 1/dt. This will be important later.
samplingRate = 1 / dt  # in Hz

Think of a discrete function as a series of weights at each of the points in `t`.  Let's look at some functions this way. Here's a gaussian profile centered at .5:

In [ ]:
f1 = exp(-.5 * (t - .5) ** 2 / (.05) ** 2)
f, ax = plt.subplots()
ax.plot(t, f1)
ax.set(xlabel='Time [s]', ylabel='Amplitude')

Pause here. The function is drawn continously: there are no gaps. That's because matplotlib interpolated between the values at the discrete time points. In reality, the function is just a bunch of heights at each of the time points. We can use the stem function to highlight this fact.

In [ ]:
f, ax = plt.subplots()
ax.stem(t, f1)

Now... that's a discrete function.Each of the points can be thought of as a weight given to the basis set of discrete time points. We represent the function as a list of these weights. Indeed, f1 is a vector.

In [ ]:
print(f1.shape)

Let's remember this function by calling it something mnemonic

In [ ]:
gaus1 = f1

Let's look at a few more functions:

In [ ]:
f, axes = plt.subplots(1, 7, figsize=(15, 3), sharex=True, sharey=True)

pulse = f2 = (t > .45) & (t <= .55)
axes[0].set(title="Pulse")
axes[0].stem(t, pulse)

# a sine 
sin1 = f3 = sin(2 * pi * 1 * (t - .5))
axes[1].set(title="Sin")
axes[1].stem(t, f3)

# the cosine
cos10 = f4 = cos(2 * pi * 10 * (t - .5))
axes[2].set(title="Cos")
axes[2].stem(t, f4)
# this looks better interpolated
axes[2].plot(t, f4)

# the sum of sine and cossine
sos = .5 * f3 + f4
axes[3].set(title="Sum of sin and cos")
axes[3].stem(t, sos)
axes[3].plot(t, sos)

# exponential
tau = .1
exp1 = f5 = exp(-(t - .5) / tau) * (t > .5)
axes[4].set(title="Exponential")
axes[4].stem(t, f5)

# delta function - take special note of this one
delt1 = f6 = t == .5
axes[5].set(title="Delta")
axes[5].stem(t, f6)

# comb(.1t) function.
# This is a sampling function or strobe function that
# turns on at a rate of 10 times per second (10 Hz).
# But it is also a bunch of delta functions.
comb10 = f7 = (t / dt % 10) == 0
axes[6].stem(t, comb10)
axes[6].set(title="Comb")

axes[0].set(xlabel='Time [s]', ylabel='Amplitude')
f.tight_layout()

Reflection point. You have yet to look at a fourier transform. You are
supposed to simply marvel at the idea of discreteness. And here is the
one thing extra that you need to think about. All of the functions above
can be thought of as sums of delta functions with appropriately chosen
weights. The most obvious example is the comb function, as we have
already noted. But all the functions are just values at the discrete time
points. So all can be thought of as delta functions. In fact, a discrete
function is exactly this: the multiplication of a continuous function by
comb(dt), where d is the gap between samples (i.e., the reciprocal of the
sampling rate). We will return to this point later.
 

One other point to make before moving on. You can think of the discrete
times as a basis set -- a list of variables or causes such that each
member of the list is  conceptualized as dimension of some (high
dimensional) space. Instead of thinking about s(t) as a 1-dimensional
function of time, we can think of it as a vector of weights (a point in a
high dimensional space). The length of the vector is the number of time
samples, which is also the dimensionality of the space. A value of a
function at one time and a value at any another time are set
independently. One way to put this is that a delta function at t=k
multiplied with any other function is 0 everywhere except at t=k. A
second point which is so trivial that it's almost confusing concerns
linearity. All of the functions we generated above are nonlinear: it is
simply not the case that $f(t_1 + t_2) = f(t_1) + f(t_2)$. However, if we think
about what we are doing with each of the weights at each sample time
point, then it is the case that linearity holds. That is, any function
can be described as a sum of delta functions at some (or all) of the
discrete sample times. 


Now, let's look at the fourier transforms of the functions above. We had
better start by defining a fourier transform. Calling it a transform
already gives away something important. The thing we're describing is not
changed; it's just expressed differently. Just like when you take a log
-- you don't change the quantity of stuff. The transform is of a function.
Now, instead of thinking of the function as returning a list of weights at the
discrete time samples, we now think of it as returning a list of "weights" on
sinsoids. We'll approach this from a couple of angles. First, let's get
the big picture.

Start with the sinusoids. They are pretty easy to conceive of as "sums of
sinusoids".  On your first pass through this, please ignore the details
of the matlab commands and pay attention to the graphs (the exception is
the line of code marked "** Look at this one **"). The left columns shows
the function of time expressed as weights at each time sample. The right
shows the corresponding fourier transform expressed as amplitudes at each
frequency. You'll want to ponder each of the 7 examples in Figure 1, but
I would encourage you to look at the two sinusoids first. The FT says
that they consist of a weight at a single frequency. The plots I made
throw away information about phase (we'll get to that in a moment). But
they make it pretty clear which frequencies comprise the 7 signals.


In [ ]:
f, axes = plt.subplots(2, 2, figsize=(7, 7), sharex="col")
nyq = 1 / (2 * dt)
fax = fftshift(fftfreq(t.size, dt))

axes[0, 0].stem(t, sin1)

axes[0, 1].stem(fax, fftshift(abs(fft(sin1))))
axes[0, 1].set(xlim=(0, nyq))

axes[1, 0].stem(t, cos10)
axes[1, 0].plot(t, cos10)

axes[1, 1].stem(fax, fftshift(abs(fft(cos10))))
axes[1, 1].set(xlim=(0, nyq))

axes[1, 0].set(xlabel='Time')
axes[1, 1].set(xlabel='Frequency')

f.tight_layout()

Now let's look at a sum of 3 sinusoids. **Look at this one.**

In [ ]:
y = sin(2 * pi * 3 * t) + .33 * sin(2 * pi * 9 * t) + .2 * sin(2 * pi * 15 * t)
f, axes = plt.subplots(1, 2, figsize=(7, 4))
axes[0].plot(t, y)
axes[1].stem(fax, fftshift(abs(fft(y))))
axes[1].set(xlim=(0, nyq))
axes[0].set(xlabel='Time')
axes[1].set(xlabel='Frequency')
f.tight_layout()

Notice that the time function is almost square wave like. It's definition makes it crystal clear that it is a sum of three sinusoids. This is pretty easy to see in the fourier transform.

Now let's look at each of the functions we made earlier. For ease of viewing, I will plot some of the time functions with lines that interpolate between the (time or frquency) points. For some functions, I use the stem command to emphasize the discreteness.   


In [ ]:
f, axes = plt.subplots(2, 7, figsize=(15, 5), sharey="row")

fs = [f1, f2, f3, f4, f5, f6, f7]
for ax_i, f_i in zip(axes.T, fs):
    ax_i[0].plot(t, f_i)
    ax_i[1].stem(fax, fftshift(abs(fft(f_i))))
    ax_i[1].set(xlim=(0, nyq))

axes[0, 0].set(xlabel="Time")
axes[1, 0].set(xlabel="Frequency")
f.tight_layout()

#### Summary of the pairs

1. A gaussian is composed of frequencies that fall off from 0 in the shape of a gaussian. This will turn out to be more true that it may seem from the figure.
2. A pulse has FT that is periodic under an envelope that falls off with frequency. This turns out to be the $\sin(\omega)/\omega$ function, called $\mathrm{sinc}(\omega)$.
3. A sine function with frequency 1 has an FT with weight only at `freq = 1`. We'll deal with phase in a moment.
4. A sinewave with frequence 10 Hz has FT with all weight at (brace yourself!) `freq = 10`.
5. An exponential looks a lot like an exponential; the weights fall off as `1 / frequency` (recall `1 / (1 + s)` from the DiffEq section).
6. A delta function is made by adding sinusoids of equal wieght at every frequency.
7. A comb function has FT that is also a comb function at the frequency of the samples and its harmonics.


Elaboration: Each frequency really gets two values or weights. Why? Because
each sinusoid has an amplitude and a phase. There are several ways to
think about this. What is probably most intuitive is just what I said
already. A sinusoid of a particular frequency needs two numbers to
describe its "intensity": amplitude and phase.  It may not be immediately
obvious that "intensity" is a good word for phase, but bear with me. The
most convenient way to think about amplitude and phase is to comprise the
sinusoid as a sum of a sine and a cosine wave. A cosine wave with 0 deg
phase is just `a * cos + 0 * sin`. A cos wave shifted by `pi / 2` radians (90 deg)
is just `0 * cos + a * sin`. A cos wave shifted by `pi / 4` radians (45 deg phase)
is `sqrt(a / 2) * cos + sqrt(a / 2) * sin`. Notice that in all these cases the
amplitude is `a`. 

Here's a demo to make this clear. What you see in the top is a sinusoid

In [ ]:
f = plt.figure(figsize=(8, 3))
gs = plt.GridSpec(1, 2, width_ratios=(3, 2))
n_steps = 36

ax_sin = f.add_subplot(gs[0])
wave, = ax_sin.plot(t, cos(t * n_steps))

ax_pol = f.add_subplot(gs[1], projection="polar")
a_all, = ax_pol.plot([0, 0], [0, 1], color="r")
a_comp, = ax_pol.plot([], [], color=".2", ls="--")
ax_pol.set(ylim=(0, 1), yticklabels=[])

def init_movie():
    return wave, a_all, a_sin, a_cos

def animate_movie(i):
    th = i * 2 * pi / n_steps
    wave.set_ydata(cos(2 * pi * t - th))
    a_all.set_xdata([th, th])
    if cos(th) >= 0 and sin(th) > 0: 
        a_comp.set_xdata([0, 0, th, pi / 2, 0])
        a_comp.set_ydata([0, cos(th), 1, sin(th), 0])
    elif cos(th) < 0 and sin(th) > 0:
        a_comp.set_xdata([0, pi / 2, th, pi, 0])
        a_comp.set_ydata([0, sin(th), 1, -cos(th), 0])
    elif cos(th) <= 0 and sin(th) < 0:
        a_comp.set_xdata([0, pi, th, 3 * pi / 2, 0])
        a_comp.set_ydata([0, -cos(th), 1, -sin(th), 0])
    else:
        a_comp.set_xdata([0, 3 * pi / 2, th, 0, 0])
        a_comp.set_ydata([0, -sin(th), 1, cos(th), 0])
    a_cos.set_ydata([0, abs(cos(th))])
    return wave, a_all, a_sin, a_cos

anim = animation.FuncAnimation(f,
                               frames=n_steps,
                               interval=100,
                               blit=True,
                               func=animate_movie,
                               init_func=init_movie)

f.tight_layout()
plt.close(f)
HTML(anim.to_html5_video())

Don't move on unless you are completely comfortable with the concept that
a sinusoid of a particular frequency and phase can be represented as the
sum of a cosine and a sine wave with different weights. When represented
on a graph with the cosine component giving the x-coordinate and the sine
component giving  the y-coordinate, the sinusoid has amplitude `sqrt(x ** 2 +
y ** 2)` and phase `arctan(y / x)`. In other words, each frequency must be
described by two numbers: either a cosine component and a sine component
or an amplitude and phase. I think most of us have better intuitions
about amplituded and phase, but the math is much easier if the two terms
are on common footing. So we usually describe the "intensity" at each
frequency by a cosine component and a sine component. These are just two
numbers or weights. Often we refer to this using a single complex number:
`x + i * y`, where x is the weight of the cosine part and y is the wieght of
the sine part. So we often refer to the cosine part as the real part of
the FT and the sine part as the imaginary part of the FT. We'll come back
to this in a moment.  It's important that you know this because when you
look at how matlab represents the value of the FT at any one frequency,
you'll see that it is a complex number!

Now, given all this talk about representing each frequency "intensity" by
two numbers, you might wonder how I plotted the graphs in Fig. 3 as
simple single-valued functions of frequency. The answer is that I used
the amplitude. That's what the 'abs' expression achieved. In fact, there
was another simplification besides. I plotted amplitudes at positive
values for frequency. In fact, the fourier transform is a function of
both positive and negative frequency.

So let's look at the fourier transforms again, but this time plot the
cosine and sine parts. I will label these the  real and imaginary parts
to get you used to the nomenclature. 

Detail. I do one sneaky thing to you here. I changed the abscissa so
that 0 shows up in the middle. I know that negative time may not mean
anything to you, but negative position (e.g., left of center) is just
as imporant as positive. I'm doing this here so that I can make a
point about symmetry later. 


In [ ]:
f, axes = plt.subplots(7, 4, figsize=(12, 12), sharex="col")

for ax_i, f_i in zip(axes, fs):

    ax_i[0].plot(t, f_i)

    a = fft(ifftshift(f_i))
    ax_i[1].plot(fax, fftshift(a.real))
    ax_i[2].plot(fax, fftshift(a.imag))
    ax_i[3].plot(fax, abs(fftshift(a)))
    plt.setp(ax_i[1:], ylim=abs(a).max()* np.array([-1.1, 1.1]))

axes[0, 0].set(title='s(t)')
axes[0, 1].set(title='Real part of FT')
axes[0, 2].set(title='Imaginary part of FT')
axes[0, 3].set(title='Amplitude of FT')
axes[-1, 0].set(xlabel='Time [s]')
plt.setp(axes[-1, 1:], xlabel='Frequency [Hz]')

f.tight_layout()

Pause and reflect. There are a few things to notice:

#### (1) Invertibility

The FT effectively decomposes our signals into sums of
sines and cosines. This process is invertible. We could start with the
fourier transform -- a list of "intensities" at each frequency -- and
convert it to a list of "intensities" as a function of time. The reason
we can get away with this is that when we listed our intensities in the
"frequency domain," we were careful to give two values: amplitude and
phase, or cosine and sine components, or real and imaginary parts.  To
ensure that the process is really invertible, we also need to make sure
that we have the right number of values in the transforms. Put simply, we
had better represent enough frequencies in the FT so that we can
reconstitute the original signal by adding together all the right
sinusoids. Let's look at how many frequencies we need.

#### (2) The nyquist frequency.

Notice the range of frequencies used to represent the FTs in this figure:

In [ ]:
min(fax), max(fax)

The maximum frequency is half the sampling rate we used to make a discrete time axis:

In [ ]:
samplingRate = 1 / dt
samplingRate

We call half of the sampling rate the nyquist frequency. This is the
highest frequency cosine wave that can be represented. This is easy to
see. To represent a sinusoid as a discrete function of time, we need a
sample in the positive going and negative going portions of the sinusoid.
The discrete FT goes from `-nyq` to `nyq - 1`. The 0 frequency (in the middle
of these graphs) is special. It represents the mean level: the
nonsinusoidal component: the `cos(0)` componenent. In fact, the traditional
way to plot the fourier transform is to begin with freq=0, increment up
to nyquist-1 and then start from -nyquist and increment up to -1 (there's
actually a good reason for this). I like seeing the 0 frequency in the
middle of graph. So I used the 'fftshift' command. You can read about
this. Try making some of the FTs without using the fftshift. What do you
see?

#### 3) Bookeeping and Hermitian symmetry.

Let's return to the theme of invertibility and do some bookeeping on the
transform? How many numbers does it take to represent the time functions
we've been dealing with? If you left `dt = 0.01` alone, the answer is 100. 

So all of the functions of time that we've considered are described by a
list this long containing the weights at each time point. What about the
FT? We just discovered that the highest frequency is the nyquist, which
is half the sampling rate. That gives us just 50 frequencies.  How do
we transform 100 values into just 50 frequencies? You know the answer: at each
frequency we need 2 numbers, a cosine and a sine component (or amplitude
and phase). Moreover, for the 0 frequency, there is just one component
becuase sin(0)=0. Since cos(0)=1, it's clear that the weight at freq=0 is
just a constant function of time. Also, at the nyquist frequency itself,
all we can have is a cosine component. To see why, recognize that we only
have enough samples in time to represent alternating positive and
negative values at adjacent time points. If you put a 0 at t=0, which you
must for sin(t), then there's nothing to alterate. Okay, cool, we've got
two components for each of the frequencies 1 to `nyquist - 1`, one weight at
`freq = 0` and one weight at `freq = nyquist`. That's 100 values in the FT, which
corresponds to 100 values in the time function. Perfect!

So what's the deal with the negative frequencies in Figure 5? The
frequency axis does not go from 0 to nyquist but from -nyquist to
nyquist-1. Doesn't that screw up our bookeeping?  Here's the answer. We
have considered the fourier transform of real valued functions of time
(left column of Fig. 5). The fourier transform returns complex numbers at
each frequency, which we can think of a sine waves (imaginary part) and
cosine waves (real part). I should say that the fourier transform can act
on complex functions of time. Instead of a  real value at each time, we
would have a complex number. We may not have any use for this, but the
full transform takes these 2-vectors (real and imaginary parts) and
transforms them to 4 values: a real and imaginary value at two
frequencies, one positive and one negative. Of course, there are half as
many frequencies as there are time points. What I'm saying is that we
actually had twice as many values in our time functions as we thought. We
used real functions of time. The extra 100 values were zeros: the weights
on the imaginary part of the signals.  So the transform is 1 to 1.

That's all fine and good in theory, but what we care about for most
applications is the fourier transform of real valued functions. That's
like saying that we know that the imaginary part of the functions in the
left column of this figure, is 0 at every time point. So, I'll ask again, why
should the FT have 200 values to represent a signal that we can describe
by 100 points in time? The answer is that the FTs of real signals really
can be represented with just 100 numbers, just like we said. You really
don't need both positive and negative frequencies. If I tell you the
weights for one of them (e.g., +5 Hz component), you know the weights of
the negative frequency (e.g., -5 Hz component). This shows up as a kind
of symmetry in the graphs in Figure 5. See if you can figure out what
this is.

I will pose this as a homework problem. Please turn in your answers to
Greg. First a reminder. An even symmetric function obeys the rule `f(x) =
f(-x)`. An example is `cos(x)`. An odd symmetric function is one that obeys
the rule `f(x) = -f(-x)`. An example is `sin(x)`.  

*QUESTION: Which of the seven functions in Fig. 5 are even? Which is odd?
Which is neither even nor odd? What do you notice about the fourier
transforms of the even and odd symmetric functions. Answer this for the
three categories: even, odd, neither even nor odd.*

----

### The complex exponential. 

I think you would not lose a lot if you skip this section, but it is
essential if you want to really grasp the math. After all, the
definition of the FT is an integral (or sum) over time of
$\exp(i \times 2 \times \pi \times w \times f(t))$, where $f(t)$ is the function of time. 

There is one other way to represent a sinusoid. It uses a complex
exponential notation. The reason this is important is that the fourier
transform is usually written using this complex exponential. This is
known as Euler's identity: $$\exp(i \times x) = \cos(x) + i \sin(x)$$


First, let's just convince ourselves that this is true.

In [ ]:
result_1 = exp(2j * pi * t)
result_2 = cos(2 * pi * t) + 1j * sin(2 * pi * t) 
np.allclose(result_1, result_2)

In [ ]:
We can see the correspondence by plotting real and imaginary parts. 
figure(6), clf, hold on
plot(t,real(result_1),'r-',t,imag(result_1),'g-')
plot(t,real(result_2),'ro',t,imag(result_1),'go')


In [ ]:
f, ax = plt.subplots()
ax.plot(t, result_1.real, "C0-")
ax.plot(t, result_1.imag, "C1-")
ax.plot(t, result_2.real, "C0.")
ax.plot(t, result_2.imag, "C1.")

Why is Euler's identity true? Here are a few intuitions. One of these
should work for you. The simplest is to recall the Taylor series
respresentation of trig functions and exp. Recall that the Taylor series
for $\cos(x)$ is: 
$$\cos(x) = 1 - \frac{x^2}{2!} + \frac{x^4}{4!} - \frac{x^6}{6!} + \frac{x^8}{8!} + \ldots$$

For sin(x) the series contains odd power terms, also alternating in sign
$$\sin(x) = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + \frac{x^9}{9!} + \ldots$$

For $\exp(x)$, we get all the terms without the sign flips:
$$\exp(x) = 1 + x + \frac{x^2}{2!} + \frac{x^3}{3!} + \frac{x^4}{4!} + \frac{x^5}{5!} + \ldots$$

So, $\exp(i\,x)$ leads to a series in which the $i^n$ terms go away for the
even powers producing alternating $-1$ for $n=2$, $+1$ for $n=4$, etc...
The $i^n$ terms leave an $i$ behind for the odd $n$ terms and also produce an
alternation of sign. 
$$
\begin{align}
\exp(i\,x)
&= 1 + i\,x - \frac{x^2}{2!} - \frac{i\,x^3}{3!} + \frac{x^4}{4!} + \frac{i\,x^5}{5!} + \ldots \\
&= \cos(x) + i\,\sin{x}
\end{align}
$$

The second intuition builds on your knowledge of differential equations.
If I tell  you that there is a function, $x(t)$, whose second derivative is
negative-proportional to itself, as in $x'' = -k\times x$,  your immediate
reaction is, "sinusoid, because that's like mass on spring". Notice that
$x(t) = \exp(i \times k \times t)$ has first derivative $i \times k \times x(t)$
and second derivative $-k^2 \times x(t)$.

Of course, I should have written $x(t) = exp(i \sqrt(k) \times t)$ and
we would get out our friend $x''=-k\times x$.

So here is the function $a * \cos(2 \times \pi \times w \times t - \phi)$. The real and imaginary parts are in dashed blue and green:

In [ ]:
w = 1
phi = pi / 4
a = 3
A = a * cos(phi)
B = a * sin(phi)
y = ((A / 2) * (exp(2j * pi * w * t) + exp(-2j * pi * w * t))
     + (B / 2) * (exp(2j * pi * w * t) - exp(-2j * pi * w * t)))

f, ax = plt.subplots()
ax.plot(t, y.real, "C2--")
ax.plot(t, y.imag, "C3--")
ax.plot(t, y.real + y.imag, "C4")

## Part II. Convolution

One of the most important uses of the FT is for computing convolution.
Convolution is an operation that looks like filtering. It takes a signal
and converts it to a new signal. Usually, we can think of the new signal
as a filtered version of the old one. We gain a deep intuition for the
process by conceptualizing the operation in the time domain and in the
frequency domain (i.e., via the fourier transform). I think this is one
of the most beautiful concepts in applied math -- and it comes up all the
time (or often if you prefer a frequency based description!).

### II.1 Convolution in the time domain

In this section, we spend a lot of time understanding convolution without
talking about FTs. But I want you to keep FTs in the back of your mind
while we do this. What you should be asking yourself is this. When
filtering looks like blurring, isn't that a lot like attenuating sharp
aspects of a signal, and isn't that a lot like attenuating high
frequencies?

Let's get started. Consider the simplest of all possible signals: a delta function,
otherwise known as a click (if it were sound pressure as a function of
time) or a line (if it were light intensity as a function of horizontal
position).

In [ ]:
time_of_impulse = .1
s1 = t == time_of_impulse
f, ax = plt.subplots()
ax.plot(t, s1)

Consider a blurring function. Let's start with one that you've grown
accustomed to: eponential decay. We'll see in a moment that this function
is causal, a useful property when it comes to time-dependent functions.

tau = .03;      % choose a time constant
b = exp(-t/tau);
plot(t,b)
set(gca,'XLim',[0 timeOfImpulse+10*tau])
xlabel('Time [out to 10\tau]')


In [ ]:
tau = .03  # choose a time constant
b = exp(-t / tau)
ax.plot(t, b)
ax.set(xlim=(0, time_of_impulse + 10 * tau))
f

Please run this next step without thinking about it. It's a trick that
allows me to make some pictures easily. You can come back to this if you
want to pursue the matrix version of convolution.

In [ ]:
from scipy.linalg import toeplitz
A = np.tril(toeplitz(b))
f, ax = plt.subplots()
ax.imshow(A)
ax.plot(A[:, -1])

### What is convolution?

It is a blurring of $s1$ by $b$. It is a new function of time that can be
thought of as a weighted sum of s1. It's actually easy to see what it is
by looking at the formula.  
$$s2(k) = \sum_d s1(d) \times b(k - d)$$

$s2$ is the result of the convolution. It is a function of time, but you've
probably noticed that the variable $t$ is not in the equation.  Actually
both $k$ and $d$ refer to time, but because we are about to shift things
around time takes on several meanings.   It's actually easier to keep
things straight if we keep $t$ out of the equation. The convolution is
defined at each time, which I will denote this by k. You will come to
view $k$ as an offset of the blur function.  The original signal remains a
function of time, but we'll use a dummy variable for time. That's what $d$
stands for: we can refer to $d$ as "dime".  There is nothing special about
$s1(d)$. It is the signal plotted as a function of "dime" instead of time.

Same thing -- no problem. What is $b(k-d)$?  It is $b$ reflected about the
y-axis and then offset by $k$ time steps to the right.  Here is what $b(k-d)$
looks like when $k=0$.

In [ ]:
d = t;
figure(1), clf
plot(-d,b)
xlabel('-dime')


In [ ]:
d = t
f, ax = plt.subplots()
ax.plot(-d, b)
ax.set(xlabel="-dime")

Here is what b(k-d) looks like for increasing values of k.

In [ ]:
f, axes = plt.subplots(10)
for k, ax in enumerate(axes):
    ax.plot(t[k] - d, b)
    ax.set(xlim=(-10 * tau, time_of_impulse + 10 * tau))
axes[-1].set(xlabel="Time [d]")
axes[0].set(title="b(k-d) for k=1 to 10")
plt.setp(axes[:-1], xticklabels=[], yticklabels=[])

It is a picture of the function reflected around the ordinate and then
offet to the right by k. Can you see this? 

Now let's generate the convolution, $s2(k)$, step of $k$ by step of $k$.
First look at the equation and convince yourself that there is one value of the
convolution produced at each offset step. Run the animation in the next
loop. You might want to put some pauses in the code. Here's what you
should see. In the top panel, $s1(d)$ is shown by a stem function in red. It is
just an impulse. The blur function, $b(k-d)$ is shown in blue. It is
stepped along as $k$ increases. The middle panel shows the product of $s1(d)$
times b(k-d). Notice that there is a value at each value for dummy time.
The bottom plot shows the sum of the products. At each time step, $k$, we
get just one value. This is the convolution, $s2(k)$. You will notice that
the sum is weighted by the width of the time bins. Why is this? What
effect does this have on the relationship between $s1$ and $s2$?


In [ ]:
f, axes = plt.subplots(3, figsize=(8, 6), sharex=True)

# TODO convolution animation

### II.2 Convolution and Fourier transforms

Fourier transforms make convolution easy. That's because convolution
between two functions of time, like $s1(t)$ and $b(t)$, is equivalent to
multiplying their fourier transforms. To be more precise, if $S_1(w)$ is the
fourier transform of $s_1(t)$ and $B(w)$ is the FT of $b(t)$, then
$$S_2(w) = S_1(w) \times B(w)$$
is the FT of $s_2(t)$.

This provides a simple recipe for computing convolutions
  1. Take the fourier transforms of the two functions you want to convolve.
  2. Multiply the FTs (there's a complex number at each frequency!)
  3. Take the inverse fourier transform

Let's do this with the exponential blurring function and $s_1$. To work with
$b$, I need it to be defined on the same time axis as $s_1$. I'll call it
`newb`.

In [ ]:
# Take the fourier transforms
B = fft(b)
S1 = fft(s1);

# Multiply them at each frequency.
S2 = B * S1

# Take the inverse fourier transform
s2 = ifft(S2)

# How does s2 look?
f, ax = plt.subplots()
ax.plot(t, s2.real)

Let's illustrate with a more realistic set of functions.